# Email Funnel Optimization Analysis

## Project Overview

This analysis examines the email marketing funnel for a SaaS product, identifying:
- Conversion rates at each funnel stage
- Key drop-off points
- Cohort-based retention patterns
- A/B test results for email campaigns
- Segment performance by source, device, and geography

**Goal:** Provide actionable recommendations to optimize the email funnel and increase paid conversions.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Plotly settings for better display
import plotly.io as pio
pio.templates.default = 'plotly_white'

## 1. Data Loading & Overview

In [ ]:
# Load datasets
users = pd.read_csv('../data/users.csv')
events = pd.read_csv('../data/email_events.csv')
conversions = pd.read_csv('../data/conversions.csv')

# Convert date columns
users['signup_date'] = pd.to_datetime(users['signup_date'])
events['event_time'] = pd.to_datetime(events['event_time'])
conversions['event_time'] = pd.to_datetime(conversions['event_time'])

print(f"Users: {len(users):,}")
print(f"Email Events: {len(events):,}")
print(f"Conversions: {len(conversions):,}")

In [ ]:
# Preview data
print("=== Users ===")
display(users.head())
print("\n=== Email Events ===")
display(events.head())
print("\n=== Conversions ===")
display(conversions.head())

## 2. Funnel Analysis

Let's analyze the conversion rates at each stage of the email funnel:

**Signup → Welcome Email → Opened → Clicked → Trial → Paid**

In [ ]:
# Calculate funnel metrics
total_users = len(users)

# Welcome emails sent
emails_sent = events[events['event_type'] == 'welcome_email_sent']['user_id'].nunique()

# Emails opened (welcome only)
emails_opened = events[
    (events['event_type'] == 'email_opened') & 
    (events['email_type'] == 'welcome')
]['user_id'].nunique()

# Emails clicked (welcome only)
emails_clicked = events[
    (events['event_type'] == 'email_clicked') & 
    (events['email_type'] == 'welcome')
]['user_id'].nunique()

# Trial started
trials_started = conversions[conversions['event_type'] == 'trial_started']['user_id'].nunique()

# Paid conversion
paid_users = conversions[conversions['event_type'] == 'paid_conversion']['user_id'].nunique()

# Create funnel dataframe
funnel_data = pd.DataFrame({
    'Stage': ['1. Signup', '2. Email Sent', '3. Email Opened', '4. Link Clicked', '5. Trial Started', '6. Paid Conversion'],
    'Users': [total_users, emails_sent, emails_opened, emails_clicked, trials_started, paid_users]
})

funnel_data['Conversion Rate (from Signup)'] = (funnel_data['Users'] / total_users * 100).round(2)
funnel_data['Stage Conversion Rate'] = (funnel_data['Users'] / funnel_data['Users'].shift(1) * 100).round(2)
funnel_data['Drop-off'] = (funnel_data['Users'].shift(1) - funnel_data['Users']).fillna(0).astype(int)
funnel_data['Drop-off Rate'] = (funnel_data['Drop-off'] / funnel_data['Users'].shift(1) * 100).round(2)

funnel_data

In [ ]:
# Funnel visualization
fig = go.Figure(go.Funnel(
    y=funnel_data['Stage'],
    x=funnel_data['Users'],
    textinfo="value+percent initial",
    textposition="inside",
    marker={"color": ["#3366CC", "#4285F4", "#5C9EFF", "#85B8FF", "#ADD1FF", "#34A853"]},
    connector={"line": {"color": "royalblue", "dash": "solid", "width": 2}}
))

fig.update_layout(
    title="Email Marketing Funnel",
    title_x=0.5,
    height=500,
    font=dict(size=14)
)

fig.show()

### Drop-off Analysis

Where are we losing the most users?

In [ ]:
# Calculate drop-offs between stages
dropoff_data = funnel_data[funnel_data['Drop-off'] > 0].copy()
dropoff_data['Stage Transition'] = [
    'Signup → Email Sent',
    'Email Sent → Opened',
    'Opened → Clicked',
    'Clicked → Trial',
    'Trial → Paid'
]

fig = px.bar(
    dropoff_data,
    x='Stage Transition',
    y='Drop-off',
    text='Drop-off Rate',
    color='Drop-off',
    color_continuous_scale='Reds'
)

fig.update_traces(texttemplate='%{text}%', textposition='outside')
fig.update_layout(
    title='Drop-off at Each Funnel Stage',
    title_x=0.5,
    xaxis_title='Funnel Transition',
    yaxis_title='Users Lost',
    height=450,
    showlegend=False
)

fig.show()

In [ ]:
# Key insight: Where is the biggest opportunity?
print("=" * 60)
print("KEY INSIGHTS: Drop-off Analysis")
print("=" * 60)

# Find biggest drop-off by absolute numbers
max_dropoff = dropoff_data.loc[dropoff_data['Drop-off'].idxmax()]
print(f"\n📉 Biggest absolute drop-off: {max_dropoff['Stage Transition']}")
print(f"   Lost {max_dropoff['Drop-off']:,} users ({max_dropoff['Drop-off Rate']}% drop-off rate)")

# Stage conversion rates
print("\n📊 Stage-by-Stage Conversion Rates:")
for i, row in funnel_data.iterrows():
    if pd.notna(row['Stage Conversion Rate']):
        print(f"   {row['Stage']}: {row['Stage Conversion Rate']}%")

## 3. Cohort Analysis

How do conversion rates vary by signup month? Are newer cohorts performing better or worse?

In [ ]:
# Create cohort analysis
def get_user_stage(user_id, events_df, conversions_df):
    """Determine the furthest stage a user reached in the funnel."""
    
    # Check paid conversion
    if user_id in conversions_df[conversions_df['event_type'] == 'paid_conversion']['user_id'].values:
        return 'Paid'
    
    # Check trial
    if user_id in conversions_df[conversions_df['event_type'] == 'trial_started']['user_id'].values:
        return 'Trial'
    
    # Check click
    if user_id in events_df[(events_df['event_type'] == 'email_clicked') & (events_df['email_type'] == 'welcome')]['user_id'].values:
        return 'Clicked'
    
    # Check open
    if user_id in events_df[(events_df['event_type'] == 'email_opened') & (events_df['email_type'] == 'welcome')]['user_id'].values:
        return 'Opened'
    
    # Check sent
    if user_id in events_df[events_df['event_type'] == 'welcome_email_sent']['user_id'].values:
        return 'Email Sent'
    
    return 'Signup Only'

# For efficiency, create sets for faster lookup
paid_users_set = set(conversions[conversions['event_type'] == 'paid_conversion']['user_id'])
trial_users_set = set(conversions[conversions['event_type'] == 'trial_started']['user_id'])
clicked_users_set = set(events[(events['event_type'] == 'email_clicked') & (events['email_type'] == 'welcome')]['user_id'])
opened_users_set = set(events[(events['event_type'] == 'email_opened') & (events['email_type'] == 'welcome')]['user_id'])
sent_users_set = set(events[events['event_type'] == 'welcome_email_sent']['user_id'])

def get_user_stage_fast(user_id):
    if user_id in paid_users_set:
        return 'Paid'
    if user_id in trial_users_set:
        return 'Trial'
    if user_id in clicked_users_set:
        return 'Clicked'
    if user_id in opened_users_set:
        return 'Opened'
    if user_id in sent_users_set:
        return 'Email Sent'
    return 'Signup Only'

users['max_stage'] = users['user_id'].apply(get_user_stage_fast)

In [ ]:
# Cohort conversion rates
cohort_stats = users.groupby('signup_month').agg(
    total_users=('user_id', 'count'),
    opened=('max_stage', lambda x: (x.isin(['Opened', 'Clicked', 'Trial', 'Paid'])).sum()),
    clicked=('max_stage', lambda x: (x.isin(['Clicked', 'Trial', 'Paid'])).sum()),
    trial=('max_stage', lambda x: (x.isin(['Trial', 'Paid'])).sum()),
    paid=('max_stage', lambda x: (x == 'Paid').sum())
).reset_index()

# Calculate rates
cohort_stats['open_rate'] = (cohort_stats['opened'] / cohort_stats['total_users'] * 100).round(2)
cohort_stats['click_rate'] = (cohort_stats['clicked'] / cohort_stats['total_users'] * 100).round(2)
cohort_stats['trial_rate'] = (cohort_stats['trial'] / cohort_stats['total_users'] * 100).round(2)
cohort_stats['conversion_rate'] = (cohort_stats['paid'] / cohort_stats['total_users'] * 100).round(2)

cohort_stats

In [ ]:
# Cohort conversion rates visualization
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=cohort_stats['signup_month'],
    y=cohort_stats['open_rate'],
    mode='lines+markers',
    name='Open Rate',
    line=dict(color='#4285F4', width=2)
))

fig.add_trace(go.Scatter(
    x=cohort_stats['signup_month'],
    y=cohort_stats['click_rate'],
    mode='lines+markers',
    name='Click Rate',
    line=dict(color='#FBBC04', width=2)
))

fig.add_trace(go.Scatter(
    x=cohort_stats['signup_month'],
    y=cohort_stats['trial_rate'],
    mode='lines+markers',
    name='Trial Rate',
    line=dict(color='#EA4335', width=2)
))

fig.add_trace(go.Scatter(
    x=cohort_stats['signup_month'],
    y=cohort_stats['conversion_rate'],
    mode='lines+markers',
    name='Paid Conversion',
    line=dict(color='#34A853', width=2)
))

fig.update_layout(
    title='Funnel Metrics by Signup Cohort',
    title_x=0.5,
    xaxis_title='Signup Month',
    yaxis_title='Conversion Rate (%)',
    height=500,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show()

In [ ]:
# Cohort heatmap - conversion rates
cohort_heatmap = cohort_stats[['signup_month', 'open_rate', 'click_rate', 'trial_rate', 'conversion_rate']].set_index('signup_month').T

fig = px.imshow(
    cohort_heatmap,
    text_auto='.1f',
    color_continuous_scale='Blues',
    aspect='auto'
)

fig.update_layout(
    title='Cohort Performance Heatmap (Conversion Rates %)',
    title_x=0.5,
    xaxis_title='Signup Month',
    yaxis_title='Metric',
    height=300
)

fig.show()

## 4. A/B Test Analysis

We ran an A/B test on the welcome email with three variants:
- **Control**: Original email
- **Variant A**: Improved subject line
- **Variant B**: Improved CTA button

Let's analyze which variant performed best.

In [ ]:
# A/B test results by variant
ab_results = users.groupby('ab_variant').agg(
    total_users=('user_id', 'count'),
    opened=('max_stage', lambda x: (x.isin(['Opened', 'Clicked', 'Trial', 'Paid'])).sum()),
    clicked=('max_stage', lambda x: (x.isin(['Clicked', 'Trial', 'Paid'])).sum()),
    converted=('max_stage', lambda x: (x == 'Paid').sum())
).reset_index()

ab_results['open_rate'] = (ab_results['opened'] / ab_results['total_users'] * 100).round(2)
ab_results['click_rate'] = (ab_results['clicked'] / ab_results['total_users'] * 100).round(2)
ab_results['conversion_rate'] = (ab_results['converted'] / ab_results['total_users'] * 100).round(2)

ab_results

In [ ]:
# Visualize A/B test results
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['Open Rate (%)', 'Click Rate (%)', 'Conversion Rate (%)']
)

colors = {'control': '#4285F4', 'variant_a': '#34A853', 'variant_b': '#EA4335'}

for i, metric in enumerate(['open_rate', 'click_rate', 'conversion_rate'], 1):
    fig.add_trace(
        go.Bar(
            x=ab_results['ab_variant'],
            y=ab_results[metric],
            marker_color=[colors[v] for v in ab_results['ab_variant']],
            text=ab_results[metric],
            textposition='outside',
            showlegend=False
        ),
        row=1, col=i
    )

fig.update_layout(
    title='A/B Test Results by Variant',
    title_x=0.5,
    height=400
)

fig.show()

In [ ]:
# Statistical significance test
def ab_test_significance(control_conversions, control_total, variant_conversions, variant_total):
    """Perform two-proportion z-test for A/B testing."""
    
    p1 = control_conversions / control_total
    p2 = variant_conversions / variant_total
    
    # Pooled proportion
    p_pool = (control_conversions + variant_conversions) / (control_total + variant_total)
    
    # Standard error
    se = np.sqrt(p_pool * (1 - p_pool) * (1/control_total + 1/variant_total))
    
    # Z-score
    z = (p2 - p1) / se if se > 0 else 0
    
    # P-value (two-tailed)
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    
    # Confidence interval for the difference
    se_diff = np.sqrt((p1 * (1-p1) / control_total) + (p2 * (1-p2) / variant_total))
    ci_lower = (p2 - p1) - 1.96 * se_diff
    ci_upper = (p2 - p1) + 1.96 * se_diff
    
    return {
        'control_rate': p1,
        'variant_rate': p2,
        'lift': (p2 - p1) / p1 * 100 if p1 > 0 else 0,
        'z_score': z,
        'p_value': p_value,
        'significant': p_value < 0.05,
        'ci_95': (ci_lower * 100, ci_upper * 100)
    }

# Test each variant against control for click rate
control_data = ab_results[ab_results['ab_variant'] == 'control'].iloc[0]

print("=" * 70)
print("A/B TEST STATISTICAL SIGNIFICANCE (Click Rate)")
print("=" * 70)

for variant in ['variant_a', 'variant_b']:
    variant_data = ab_results[ab_results['ab_variant'] == variant].iloc[0]
    
    result = ab_test_significance(
        control_data['clicked'], control_data['total_users'],
        variant_data['clicked'], variant_data['total_users']
    )
    
    print(f"\n{variant.upper()} vs CONTROL:")
    print(f"  Control click rate:  {result['control_rate']*100:.2f}%")
    print(f"  Variant click rate:  {result['variant_rate']*100:.2f}%")
    print(f"  Lift:                {result['lift']:+.1f}%")
    print(f"  P-value:             {result['p_value']:.4f}")
    print(f"  95% CI:              ({result['ci_95'][0]:+.2f}%, {result['ci_95'][1]:+.2f}%)")
    print(f"  Significant (α=0.05): {'✅ YES' if result['significant'] else '❌ NO'}")

## 5. Segmentation Analysis

Let's analyze how different user segments perform in the funnel.

In [ ]:
# Segment analysis by traffic source
source_stats = users.groupby('source').agg(
    total_users=('user_id', 'count'),
    opened=('max_stage', lambda x: (x.isin(['Opened', 'Clicked', 'Trial', 'Paid'])).sum()),
    clicked=('max_stage', lambda x: (x.isin(['Clicked', 'Trial', 'Paid'])).sum()),
    converted=('max_stage', lambda x: (x == 'Paid').sum())
).reset_index()

source_stats['open_rate'] = (source_stats['opened'] / source_stats['total_users'] * 100).round(2)
source_stats['click_rate'] = (source_stats['clicked'] / source_stats['total_users'] * 100).round(2)
source_stats['conversion_rate'] = (source_stats['converted'] / source_stats['total_users'] * 100).round(2)

source_stats = source_stats.sort_values('conversion_rate', ascending=False)
source_stats

In [ ]:
# Source performance visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Users by Source', 'Conversion Rate by Source'],
    specs=[[{"type": "pie"}, {"type": "bar"}]]
)

# Pie chart for volume
fig.add_trace(
    go.Pie(
        labels=source_stats['source'],
        values=source_stats['total_users'],
        textinfo='percent+label',
        hole=0.3
    ),
    row=1, col=1
)

# Bar chart for conversion
fig.add_trace(
    go.Bar(
        x=source_stats['source'],
        y=source_stats['conversion_rate'],
        marker_color='#4285F4',
        text=source_stats['conversion_rate'].apply(lambda x: f'{x}%'),
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    title='Traffic Source Analysis',
    title_x=0.5,
    height=450,
    showlegend=False
)

fig.show()

In [ ]:
# Segment analysis by device
device_stats = users.groupby('device').agg(
    total_users=('user_id', 'count'),
    opened=('max_stage', lambda x: (x.isin(['Opened', 'Clicked', 'Trial', 'Paid'])).sum()),
    clicked=('max_stage', lambda x: (x.isin(['Clicked', 'Trial', 'Paid'])).sum()),
    converted=('max_stage', lambda x: (x == 'Paid').sum())
).reset_index()

device_stats['open_rate'] = (device_stats['opened'] / device_stats['total_users'] * 100).round(2)
device_stats['click_rate'] = (device_stats['clicked'] / device_stats['total_users'] * 100).round(2)
device_stats['conversion_rate'] = (device_stats['converted'] / device_stats['total_users'] * 100).round(2)

print("Performance by Device:")
device_stats

In [ ]:
# Device performance visualization
fig = go.Figure()

metrics = ['open_rate', 'click_rate', 'conversion_rate']
colors = ['#4285F4', '#FBBC04', '#34A853']

for metric, color in zip(metrics, colors):
    fig.add_trace(go.Bar(
        name=metric.replace('_', ' ').title(),
        x=device_stats['device'],
        y=device_stats[metric],
        marker_color=color
    ))

fig.update_layout(
    title='Funnel Metrics by Device Type',
    title_x=0.5,
    xaxis_title='Device',
    yaxis_title='Rate (%)',
    barmode='group',
    height=400
)

fig.show()

In [ ]:
# Segment analysis by country (top 5)
country_stats = users.groupby('country').agg(
    total_users=('user_id', 'count'),
    opened=('max_stage', lambda x: (x.isin(['Opened', 'Clicked', 'Trial', 'Paid'])).sum()),
    clicked=('max_stage', lambda x: (x.isin(['Clicked', 'Trial', 'Paid'])).sum()),
    converted=('max_stage', lambda x: (x == 'Paid').sum())
).reset_index()

country_stats['open_rate'] = (country_stats['opened'] / country_stats['total_users'] * 100).round(2)
country_stats['click_rate'] = (country_stats['clicked'] / country_stats['total_users'] * 100).round(2)
country_stats['conversion_rate'] = (country_stats['converted'] / country_stats['total_users'] * 100).round(2)

country_stats = country_stats.sort_values('conversion_rate', ascending=False)

# Visualization
fig = px.bar(
    country_stats,
    x='country',
    y='conversion_rate',
    color='conversion_rate',
    color_continuous_scale='Greens',
    text='conversion_rate'
)

fig.update_traces(texttemplate='%{text}%', textposition='outside')
fig.update_layout(
    title='Paid Conversion Rate by Country',
    title_x=0.5,
    xaxis_title='Country',
    yaxis_title='Conversion Rate (%)',
    height=400,
    showlegend=False
)

fig.show()

## 6. Revenue Analysis

In [ ]:
# Revenue by plan
paid_conversions = conversions[conversions['event_type'] == 'paid_conversion'].copy()

if 'revenue' not in paid_conversions.columns:
    plan_prices = {'basic': 29, 'pro': 79, 'enterprise': 199}
    paid_conversions['revenue'] = paid_conversions['plan'].map(plan_prices)

revenue_by_plan = paid_conversions.groupby('plan').agg(
    customers=('user_id', 'count'),
    total_revenue=('revenue', 'sum'),
    avg_revenue=('revenue', 'mean')
).reset_index()

revenue_by_plan['revenue_share'] = (revenue_by_plan['total_revenue'] / revenue_by_plan['total_revenue'].sum() * 100).round(1)

revenue_by_plan

In [ ]:
# Revenue visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Customers by Plan', 'Revenue by Plan'],
    specs=[[{"type": "pie"}, {"type": "pie"}]]
)

fig.add_trace(
    go.Pie(
        labels=revenue_by_plan['plan'],
        values=revenue_by_plan['customers'],
        name='Customers',
        marker_colors=['#4285F4', '#34A853', '#FBBC04']
    ),
    row=1, col=1
)

fig.add_trace(
    go.Pie(
        labels=revenue_by_plan['plan'],
        values=revenue_by_plan['total_revenue'],
        name='Revenue',
        marker_colors=['#4285F4', '#34A853', '#FBBC04']
    ),
    row=1, col=2
)

fig.update_layout(
    title='Revenue Distribution by Plan',
    title_x=0.5,
    height=400
)

fig.show()

In [ ]:
# Revenue by source
paid_with_source = paid_conversions.merge(users[['user_id', 'source']], on='user_id')

revenue_by_source = paid_with_source.groupby('source').agg(
    customers=('user_id', 'count'),
    total_revenue=('revenue', 'sum')
).reset_index()

# Add total users to calculate ROI
source_users = users.groupby('source')['user_id'].count().reset_index()
source_users.columns = ['source', 'total_signups']

revenue_by_source = revenue_by_source.merge(source_users, on='source')
revenue_by_source['revenue_per_signup'] = (revenue_by_source['total_revenue'] / revenue_by_source['total_signups']).round(2)
revenue_by_source = revenue_by_source.sort_values('revenue_per_signup', ascending=False)

# Visualization
fig = px.bar(
    revenue_by_source,
    x='source',
    y='revenue_per_signup',
    color='revenue_per_signup',
    color_continuous_scale='Greens',
    text='revenue_per_signup'
)

fig.update_traces(texttemplate='$%{text}', textposition='outside')
fig.update_layout(
    title='Revenue per Signup by Traffic Source',
    title_x=0.5,
    xaxis_title='Traffic Source',
    yaxis_title='Revenue per Signup ($)',
    height=400,
    showlegend=False
)

fig.show()

## 7. Summary & Recommendations

In [ ]:
print("="*70)
print("EMAIL FUNNEL OPTIMIZATION - EXECUTIVE SUMMARY")
print("="*70)

print("\n📊 FUNNEL OVERVIEW")
print("-"*50)
for _, row in funnel_data.iterrows():
    print(f"{row['Stage']}: {row['Users']:,} users ({row['Conversion Rate (from Signup)']}%)")

print("\n📉 KEY DROP-OFF POINTS")
print("-"*50)
print(f"1. Email Sent → Opened: {funnel_data.iloc[2]['Drop-off Rate']}% drop-off (need better subject lines)")
print(f"2. Opened → Clicked: {funnel_data.iloc[3]['Drop-off Rate']}% drop-off (need better CTAs)")
print(f"3. Clicked → Trial: {funnel_data.iloc[4]['Drop-off Rate']}% drop-off (landing page optimization)")

print("\n🏆 TOP PERFORMING SEGMENTS")
print("-"*50)
best_source = source_stats.iloc[0]
print(f"Best traffic source: {best_source['source']} ({best_source['conversion_rate']}% conversion)")
best_device = device_stats.sort_values('conversion_rate', ascending=False).iloc[0]
print(f"Best device: {best_device['device']} ({best_device['conversion_rate']}% conversion)")
best_country = country_stats.iloc[0]
print(f"Best country: {best_country['country']} ({best_country['conversion_rate']}% conversion)")

print("\n🧪 A/B TEST WINNER")
print("-"*50)
winner = ab_results.sort_values('click_rate', ascending=False).iloc[0]
print(f"Best variant: {winner['ab_variant']} ({winner['click_rate']}% click rate)")

print("\n💰 REVENUE INSIGHTS")
print("-"*50)
total_revenue = paid_conversions['revenue'].sum()
print(f"Total revenue: ${total_revenue:,.0f}")
print(f"Paying customers: {len(paid_conversions)}")
print(f"ARPU: ${total_revenue/len(paid_conversions):.2f}")

## Recommendations

Based on the analysis, here are the key recommendations:

### 1. Improve Email Open Rates
- **Current open rate:** ~45%
- **Action:** A/B test more subject lines. Variant A showed 15% lift.
- **Expected impact:** +2-3% overall conversion

### 2. Optimize Click-Through Rate
- **Current CTR:** ~12% of opens
- **Action:** Implement Variant B's CTA design (30% lift in clicks)
- **Expected impact:** +20-30% more trials

### 3. Focus on High-Value Segments
- **Referral traffic** has highest conversion - invest in referral program
- **Desktop users** convert better - optimize mobile experience
- **Germany (DE)** leads in conversion - consider localization

### 4. Reduce Friction in Trial Signup
- 65% of clickers don't start trial
- **Action:** Simplify signup form, add social login
- **Action:** Add exit-intent popup with special offer

### 5. Improve Trial-to-Paid Conversion
- 75% of trial users don't convert
- **Action:** Improve onboarding sequence
- **Action:** Add mid-trial check-in emails
- **Action:** Offer annual discount at trial end

---

**Projected Impact:** If we improve each stage by 10%, the overall conversion rate would increase from 0.8% to approximately 1.3% (+62% improvement).